In [1]:
import requests
from tqdm import tqdm
import pandas as pd

In [2]:
NO_TRIES = 10
NO_N = 15

In [14]:
address = "http://localhost:8413/done"

def single_test(data, NO_TRIES):
    res = []
    for i in tqdm(range(NO_TRIES)):
        r = requests.post(url=address, data = data)
        res.append((r.elapsed, len(r.text)))
    return res

def test(rules, NO_N, NO_TRIES, filename):
    TESTED_N = [2**x for x in range(NO_N)]
    
    res = {}
    for n in TESTED_N:
        data = {
            "terminals": "a",
            "nonterminals": "<a>",
            "head": "<a>",
            "rules": rules,
            "input": "a " * n,
            "visualization": "none"
        }
        res[n] = single_test(data, NO_TRIES)

    times = {}
    lens = {}
    for k, v in res.items():
        times[k] = [x[0].seconds + .000001 * x[0].microseconds for x in v]
        lens[k] = [x[1] for x in v]
        
    df_times = pd.DataFrame(times).T
    df_times["mean"] = df_times.mean(1)
    df_times["std"] = df_times.std(1)
    df_times.to_pickle(filename + "_times.pkl")
                    
    df_lens = pd.DataFrame(lens).T
    df_lens["mean"] = df_lens.mean(1)
    df_lens["std"] = df_lens.std(1)
    df_times.to_pickle(filename + "_times.pkl")
    
    return df_times, df_lens

In [17]:
df = test("<a> ::= <a> a | a", NO_N, NO_TRIES, "linear")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 367.02it/s]


In [22]:
df = test("<a> ::= <a> <a> | a", 4, NO_TRIES, "linear")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 21.28it/s]
